# شناسایی مشکلات در یک مجموعه داده متنی با &#8235; Cleanlab

نویسنده: &#8235;[Aravind Putrevu](https://huggingface.co/aravindputrevu)

[Mansoor Nabawi](https://github.com/Mansoorinho) :مترجم
 

در این آموزش  شروع سریع 5 دقیقه ای, ما از &#8235; Cleanlab برای شناسایی مشکلات مختلف در یک مجموعه داده دسته بندی قصد (intent classifcation dataset) که از درخواست های خدمات مشتری (متن) در یک بانک آنلاین تشکیل شده است استفاده میکنیم. ما زیرمجموعه‌ای از [Banking77-OOS Dataset](https://arxiv.org/abs/2106.04564) را در نظر می‌گیریم که شامل 1000 درخواست خدمات مشتری است که بر اساس هدفشان در 10 دسته طبقه‌بندی شده اند. (شما میتوانید همین کد را روی هر مجموعه داده دسته بندی اجرا کنید). [Cleanlab](https://github.com/cleanlab/cleanlab) به‌طور خودکار نمونه‌های بد را در مجموعه داده‌های ما شناسایی می‌کند، از جمله داده‌های دارای برچسب اشتباه، نمونه‌ داده های پرت (outliers)، یا نمونه‌های مبهم دیگر.  قبل از اینکه به مدل‌سازی داده‌هایتان بپردازید! ,فیلتر کردن یا تصحیح چنین مثال‌های بد را در نظر بگیرید.

**:مروری بر کارهایی که در این آموزش انجام خواهیم داد**

- از یک مدل ترنسفورمر&#8235; (transformer) از قبل آموزش دیده شده برای استخراج embedding های متن از درخواست های خدمات مشتری استفاده کنید.

- یک مدل ساده &#8235; (Logistic Regression) بر روی embedding های متن آموزش دهید تا احتمالات خارج از نمونه را پیش بینی کند.

- حسابرسی &#8235;`Datalab` را در Cleanlab با این پیش بینی ها و embedding ها اجرا کنید تا مشکلاتی مانند : مسایل برچسب زنی, داده های پرت (outliers) و تقریبا تکراری را در مجموعه داده شناسایی شوند.

## شروع سریع

    
 آیا قبلا &#8235;`pred_probs` (out-of-sample خارج از نمونه) از یک مدل آموزش داده شده روی مجموعه‌ای از برچسب‌ها موجود دارید؟ شاید شما هم تعدادی `features` عددی داشته باشید؟ کد زیر را اجرا کنید تا هر گونه خطای برچسب احتمالی را در مجموعه داده خود بیابید.

**توجه:** اگر کد خود را روی &#8235;Colab اجرا می کنید، ممکن است بخواهید از GPU استفاده کنید (برای اینکار انتخاب کنید: Runtime > Change runtime type > Hardware accelerator > GPU)

In [ ]:
from cleanlab import Datalab

lab = Datalab(data=your_dataset, label_name="column_name_of_labels")
lab.find_issues(pred_probs=your_pred_probs, features=your_features)

lab.report()
lab.get_issues()


## کتابخانه های لازم را نصب کنید


شما میتوانید از	&#8235; `pip` برای نصب تمامی بسته/کتابخانه های لازم برای این آموزش  طبق موارد زیر استفاده کنید:

In [ ]:
!pip install -U scikit-learn sentence-transformers datasets
!pip install -U "cleanlab[datalab]"

In [ ]:
import re
import string
import pandas as pd
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

from cleanlab import Datalab

In [ ]:
import random
import numpy as np

pd.set_option("display.max_colwidth", None)

SEED = 123456  # for reproducibility
np.random.seed(SEED)
random.seed(SEED)

## مجموعه داده متنی را بارگیری و قالب بندی کنید

In [ ]:
from datasets import load_dataset

dataset = load_dataset("PolyAI/banking77", split="train")
data = pd.DataFrame(dataset[:1000])
data.head()

,text,label
0,I am still waiting on my card?,11
1,What can I do if my card still hasn't arrived after 2 weeks?,11
2,I have been waiting over a week. Is the card still coming?,11
3,Can I track my card while it is in the process of delivery?,11
4,"How do I know if I will get my card, or if it is lost?",11


In [ ]:
raw_texts, labels = data["text"].values, data["label"].values
num_classes = len(set(labels))

print(f"This dataset has {num_classes} classes.")
print(f"Classes: {set(labels)}")

This dataset has 7 classes.
Classes: {32, 34, 36, 11, 13, 46, 17}


مثال 	&#8235;i-م را در این مجموعه داده مشاهده کنید:

In [ ]:
i = 1  # change this to view other examples from the dataset
print(f"Example Label: {labels[i]}")
print(f"Example Text: {raw_texts[i]}")

Example Label: 11
Example Text: What can I do if my card still hasn't arrived after 2 weeks?


داده ها به صورت دو آرایه 	&#8235;numpy ذخیره می شوند:

1. 	&#8235; `raw_texts` درخواست های خدمات مشتری را در قالب متن ذخیره می کند
2.  &#8235; `labels` دسته های قصد (برچسب‌ها) را برای هر مثال ذخیره می‌ کند

<div class="alert alert-info">
&#8235; داده خود را بیاورید ؟Bringing Your Own Data (BYOD)

شما به راحتی می توانید مجموعه داده های متنی خود را جایگزین موارد فوق کنید و با بقیه آموزش ادامه دهید. 

</div>

سپس رشته های متنی را به بردارهایی تبدیل می کنیم که به عنوان ورودی برای مدل های &#8235; ML ما مناسب تر هستند.

ما از نمایش های عددی از یک مدل ترانسفورمر از پیش آموزش دیده به عنوان &#8235;embeddings متن خود استفاده خواهیم کرد. کتابخانه &#8235;[Sentence Transformers](https://huggingface.co/docs/hub/sentence-transformers) روش های ساده ای را برای محاسبه این &#8235; embedding ها برای داده های متنی ارائه می دهد. در اینجا، مدل &#8235; `electra-small-discriminator` از پیش آموزش دیده را بارگیری می کنیم و سپس داده های خود را از طریق شبکه اجرا می کنیم تا یک &#8235; embedding برداری از هر نمونه استخراج کنیم.

In [ ]:
transformer = SentenceTransformer('google/electra-small-discriminator')
text_embeddings = transformer.encode(raw_texts)

مدل &#8235; ML بعدی ما مستقیماً بر روی عناصر &#8235; `text_embeddings` عمل می‌کند تا درخواست‌های خدمات مشتری را طبقه‌بندی کند.

## یک مدل دسته بندی تعریف کنید و احتمالات پیش بینی شده خارج از نمونه را محاسبه کنید

یک روش معمول برای استفاده از شبکه های از پیش آموزش دیده&#8235; (pretrained networks) برای یک کار دسته بندی خاص، اضافه کردن یک لایه خروجی خطی و تنظیم دقیق پارامترهای شبکه بر روی داده های جدید است. با این حال این می تواند از نظر محاسباتی فشرده باشد. از طرف دیگر، می‌توانیم وزن‌های از پیش آموزش‌ دیده‌شده&#8235; (pretrained weights) شبکه را ثابت/منجمد &#8235;(freeze) کنیم و فقط لایه خروجی را بدون تکیه بر GPU (ها) آموزش دهیم. در اینجا ما این کار را به راحتی با قرار دادن یک مدل خطی &#8235; scikit-learn در بالای &#8235;embedding های استخراج شده انجام می دهیم.

برای شناسایی مشکلات برچسب،  &#8235;cleanlab به یک پیش‌بینی احتمالی از مدل شما برای هر نقطه داده نیاز دارد. با این حال، این پیش‌بینی‌ها برای نقاط داده‌ای که مدل قبلاً روی آن‌ها آموزش داده شده، &#8235; _overfit_ میباشد (و در نتیجه غیرقابل اعتماد) خواهد بود. &#8235; cleanlab فقط برای استفاده با احتمالات کلاس پیش بینی شده &#8235; **out-of-sample** **خارج از نمونه** در نظر گرفته شده. یعنی در نقاطی که از مدل در هنگام آموزش دادن کنار گذاشته شده اند.


در اینجا ما با استفاده از یک مدل &#8235; Logistic Regression با &#8235; cross-validation، احتمالات کلاس پیش‌بینی‌شده خارج از نمونه را برای هر نمونه در مجموعه داده خود به دست می‌آوریم.
مطمئن شوید که ستون‌های &#8235;`pred_probs` شما با توجه به ترتیب کلاس‌ها به درستی مرتب شده‌اند، که برای &#8235; Datalab این است: از نظر واژگانی بر اساس نام کلاس مرتب شده‌اند.

In [ ]:
model = LogisticRegression(max_iter=400)

pred_probs = cross_val_predict(model, text_embeddings, labels, method="predict_proba")

## از &#8235; Cleanlab برای پیدا کردن مشکلات در مجموعه داده تان استفاده کنید؟

با توجه به &#8235; feature embedding های داده شده و احتمالات کلاس پیش‌بینی‌شده (خارج از نمونه) به‌دست‌آمده از هر مدلی که دارید، &#8235; cleanlab می‌تواند به سرعت به شما در شناسایی نمونه‌های با کیفیت پایین در مجموعه داده‌تان کمک کند.

در اینجا، ما از &#8235;`Datalab` در  &#8235;Cleanlab برای یافتن مشکلات در داده‌های خود استفاده می‌کنیم. Datalab راه های مختلفی برای بارگذاری داده ها ارائه می دهد. ما به سادگی &#8235; feature های آموزشی و برچسب های دارای اختلال را در یک &#8235; dictionary قرار می دهیم.

In [ ]:
data_dict = {"texts": raw_texts, "labels": labels}

تنها چیزی که برای بررسی داده های شما نیاز است، فراخوانی &#8235;`()find_issues` است. ما احتمالات پیش‌بینی‌شده و &#8235; feature embedding های به‌دست‌آمده در بالا را به آن میدهیم، اما شما لزوماً نیازی به ارائه همه این اطلاعات بسته به نوع مسائلی که به آن‌ها علاقه دارید ندارید. هرچه شما ورودی بیشتری فراهیم کنید &#8235; `Datalab` میتواند مشکلات و مسایل بیشتری را در داده شما شناسایی کند. استفاده از یک مدل بهتر برای تولید این ورودی‌ها، تضمین می‌کند که &#8235; cleanlab با دقت بیشتری مسائل را تخمین می‌زند.

In [ ]:
lab = Datalab(data_dict, label_name="labels")
lab.find_issues(pred_probs=pred_probs, features=text_embeddings)

:خروجی میتواند به صورت زیر باشد

```bash
Finding null issues ...
Finding label issues ...
Finding outlier issues ...
Fitting OOD estimator based on provided features ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Finding class_imbalance issues ...
Finding underperforming_group issues ...

Audit complete. 62 issues found in the dataset.
```

: پس از تکمیل رسیدگی، یافته ها را با استفاده از روش &#8235; `report` مرور کنید

In [ ]:
lab.report()

Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
       outlier          37
near_duplicate          14
         label          10
       non_iid           1

Dataset Information: num_examples: 1000, num_classes: 7


---------------------- outlier issues ----------------------

About this issue:
	Examples that are very different from the rest of the dataset 
    (i.e. potentially out-of-distribution or rare/anomalous instances).
    

Number of examples with this issue: 37
Overall dataset quality in terms of this issue: 0.3671

Examples representing most severe instances of this issue:
     is_outlier_issue  outlier_score
791              True       0.024866
601              True       0.031162
863              True       0.060738
355              True       0.064199
157              True       0.065075


------------------ near_duplicate issues -------------------

About this issue:
	A (near) duplicate issue refers to two or more example

### مشکلات برچسب زنی

این گزارش نشان می دهد که &#8235; cleanlab بسیاری از مشکلات برچسب را در مجموعه داده ما شناسایی کرده است. ما می‌توانیم ببینیم کدام نمونه‌ها به‌عنوان احتمال اشتباه برچسب‌گذاری شده‌اند و امتیاز کیفیت برچسب برای هر نمونه با استفاده از متد &#8235; `get_issues`، مشخص کردن &#8235; `label` به‌عنوان آرگومان برای تمرکز بر روی مسائل برچسب در داده‌ها.

In [ ]:
label_issues = lab.get_issues("label")
label_issues.head()

,is_label_issue,label_score,given_label,predicted_label
0,False,0.903926,11,11
1,False,0.860544,11,11
2,False,0.658309,11,11
3,False,0.697085,11,11
4,False,0.434934,11,11


| | is_label_issue | label_score | given_label | predicted_label |
|----------------|-------------|-------------|-----------------|-----------------|
| 0              | False       | 0.903926    | 11              | 11 |
| 1              | False       | 0.860544    | 11              | 11 |
| 2              | False       | 0.658309    | 11              | 11 |
| 3              | False       | 0.697085    | 11              | 11 |
| 4              | False       | 0.434934    | 11              | 11 |


این روش یک دیتافریم حاوی امتیاز کیفیت برچسب &#8235;(label quality score) را برای هر مثال برمی گرداند. این نمرات عددی بین 0 و 1 قرار دارند، که در آن نمرات پایین تر نشان می دهد که نمونه هایی با احتمال بیشتری به اشتباه برچسب زده می شوند. این دیتافریم همچنین حاوی یک ستون بولین است که مشخص می کند آیا هر نمونه دارای مشکل برچسب است یا خیر (که نشان می دهد احتمالاً برچسب اشتباهی دارد).

ما می‌توانیم زیرمجموعه‌ای از نمونه‌های علامت گذاری شده با مشکلات برچسب را دریافت کنیم، و همچنین بر اساس امتیاز کیفیت برچسب مرتب‌ سازی کنیم تا شاخص‌های 5 نمونه با احتمال اشتباه برچسب‌گذاری شده را در مجموعه داده ما پیدا کنیم. 

In [ ]:
identified_label_issues = label_issues[label_issues["is_label_issue"] == True]
lowest_quality_labels = label_issues["label_score"].argsort()[:5].to_numpy()

print(
    f"cleanlab found {len(identified_label_issues)} potential label errors in the dataset.\n"
    f"Here are indices of the top 5 most likely errors: \n {lowest_quality_labels}"
)

cleanlab found 10 potential label errors in the dataset.
Here are indices of the top 5 most likely errors: 
 [379 100 300 485 159]


بیایید برخی از محتمل ترین خطاهای برچسب را مرور کنیم.

در اینجا ما 5 نمونه برتر شناسایی شده به عنوان محتمل ترین خطاهای برچسب در مجموعه داده را همراه با برچسب (اصلی) آنها و یک برچسب جایگزین پیشنهادی از &#8235;cleanlab نشان می دهیم.

In [ ]:
data_with_suggested_labels = pd.DataFrame(
    {"text": raw_texts, "given_label": labels, "suggested_label": label_issues["predicted_label"]}
)
data_with_suggested_labels.iloc[lowest_quality_labels]

,text,given_label,suggested_label
379,Is there a specific source that the exchange rate for the transfer I'm planning on making is pulled from?,32,11
100,can you share card tracking number?,11,36
300,"If I need to cash foreign transfers, how does that work?",32,46
485,Was I charged more than I should of been for a currency exchange?,17,34
159,Is there any way to see my card in the app?,13,11


:خروجی دستور بالا به شکل زیر است
  
|      | text                                                                                                      | given_label    | suggested_label |
|------|-----------------------------------------------------------------------------------------------------------|----------------|-----------------|
| 379  | Is there a specific source that the exchange rate for the transfer I'm planning on making is pulled from? | 32             | 11              |
| 100  | can you share card tracking number?                                                                       | 11             | 36              |
| 300  | If I need to cash foreign transfers, how does that work?                                                  | 32             | 46              |
| 485  | Was I charged more than I should of been for a currency exchange?                                         | 17             | 34              |
| 159  | Is there any way to see my card in the app?                                                               | 13             | 11              |


اینها خطاهای برچسب زنی بسیار واضحی هستند که &#8235;cleanlab در این داده ها شناسایی کرده است! توجه داشته باشید که &#8235;`given_label` به درستی قصد این درخواست‌ها را منعکس نمی‌کند، هر کسی که این مجموعه داده را تولید کرده است اشتباهات زیادی مرتکب شده است که قبل از مدل‌سازی داده‌ها باید به آنها رسیدگی شود.

### مشکلات داده های پرت/دورافتاده&#8235; Outlier

بر اساس این گزارش، مجموعه داده ما حاوی برخی موارد داده پرت است.
از طریق &#8235;`get_issues` می‌توانیم ببینیم کدام نمونه‌ها پرت هستند (و یک امتیاز کیفی عددی که مشخص می‌کند هر نمونه چقدر معمولی به نظر می‌رسد). ما &#8235; DataFrame دیتافریم به دست آمده را بر اساس امتیاز کیفیت داده پرتِ &#8235;cleanlab مرتب می کنیم تا دورافتاده ترین داده های پرت را در مجموعه داده خود ببینیم.

In [ ]:
outlier_issues = lab.get_issues("outlier")
outlier_issues.sort_values("outlier_score").head()

,is_outlier_issue,outlier_score
791,True,0.024866
601,True,0.031162
863,True,0.060738
355,True,0.064199
157,True,0.065075


:خروجی به شکل زیر خواهد بود

|   | is_outlier_issue | outlier_score |
|---| ----------------|---------------|
| 791 | True             | 0.024866      |
| 601 | True             | 0.031162      |
| 863 | True             | 0.060738      |
| 355 | True             | 0.064199      |
| 157 | True             | 0.065075      |

In [ ]:
lowest_quality_outliers = outlier_issues["outlier_score"].argsort()[:5]

data.iloc[lowest_quality_outliers]

,text,label
791,withdrawal pending meaning?,46
601,$1 charge in transaction.,34
863,My atm withdraw is stillpending,46
355,explain the interbank exchange rate,32
157,"lost card found, want to put it back in app",13


:یک نمونه خروجی برای پایین ترین کیفیت های داده پرت به شکل زیر است

|index|text|label|
|---|---|---|
|791|withdrawal pending meaning?|46|
|601|$1 charge in transaction\.|34|
|863|My atm withdraw is stillpending|46|
|355|explain the interbank exchange rate|32|
|157|lost card found, want to put it back in app|13|


می بینیم که &#8235;cleanlab ورودی هایی را در این مجموعه داده شناسایی کرده است که به نظر نمی رسد درخواست مشتری مناسبی باشد. به نظر می‌رسد موارد پرت در این مجموعه داده‌ها درخواست‌های مشتری خارج از محدوده و سایر متن‌های بی‌معنی است که برای دسته بندی قصد منطقی نیست. به دقت در نظر بگیرید که آیا چنین موارد پرت ممکن است بر مدل‌سازی داده‌های شما تأثیر منفی بگذارد یا خیر، و در صورت وجود، حذف آنها از مجموعه داده را در نظر بگیرید.

### مشکلات تقریبا تکراری

بر اساس این گزارش، مجموعه داده ما شامل مجموعه‌ای از نمونه‌های تقریبا تکراری است.
از طریق &#8235;`get_issues` می‌توانیم ببینیم کدام نمونه‌ها (تقریباً) تکراری هستند (و یک امتیاز کیفی عددی که مشخص می‌کند چقدر هر نمونه با نزدیک‌ترین همسایه‌اش در مجموعه داده متفاوت است). ما DataFrame حاصل را بر اساس نمره کیفیت تقریبا تکراریِ &#8235;cleanlab مرتب می‌کنیم تا نمونه‌های متنی را در مجموعه داده خود ببینیم که تقریباً تکراری هستند.

In [ ]:
duplicate_issues = lab.get_issues("near_duplicate")
duplicate_issues.sort_values("near_duplicate_score").head()

,is_near_duplicate_issue,near_duplicate_score,near_duplicate_sets,distance_to_nearest_neighbor
459,True,0.009544,[429],0.000566
429,True,0.009544,[459],0.000566
501,True,0.046044,"[412, 517]",0.002781
412,True,0.046044,[501],0.002781
698,True,0.054626,[607],0.003314


نتایج بالا نشان می‌دهد که &#8235;cleanlab کدام نمونه‌ها را تقریباً تکراری در نظر می‌گیرد (ردیف‌هایی که &#8235;`is_near_duplicate_issue == True` است). در اینجا، می بینیم که مثال های 459 و 429 تقریباً تکراری هستند، همانطور که نمونه های 501 و 412 هستند.

بیایید این نمونه ها را ببینیم تا ببینیم چقدر شبیه هستند.

In [ ]:
data.iloc[[459, 429]]

,text,label
459,I purchased something abroad and the incorrect exchange rate was applied.,17
429,I purchased something overseas and the incorrect exchange rate was applied.,17


: نمونه خروجی

|index|text|label|
|---|---|---|
|459|I purchased something abroad and the incorrect exchange rate was applied\.|17|
|429|I purchased something overseas and the incorrect exchange rate was applied\.|17|

In [ ]:
data.iloc[[501, 412]]

,text,label
501,The exchange rate you are using is really bad.This can't be the official interbank exchange rate.,17
412,The exchange rate you are using is bad.This can't be the official interbank exchange rate.,17


: نمونه خروجی

|index|text|label|
|---|---|---|
|501|The exchange rate you are using is really bad\.This can't be the official interbank exchange rate\.|17|
|412|The exchange rate you are using is bad\.This can't be the official interbank exchange rate\.|17|

می بینیم که این دو مجموعه درخواست در واقع بسیار شبیه یکدیگر هستند! نمونه های تقریباً تکراری در یک مجموعه داده ممکن است تأثیرات ناخواسته ای روی مدل ها داشته باشد و در مورد تقسیم آنها در مجموعه های آموزشی&#8235; (training)/تست &#8235;(test) محتاط باشید. درباره مدیریت نزدیک به موارد تکراری در مجموعه داده از &#8235;[the FAQ](../faq.html#How-to-handle-near-duplicate-data-identified-by-cleanlab?) بیشتر بیاموزید.

### مشکلات توزیع نا-یکسان و نا-مستقل &#8235;Non-IID (انحراف داده ها)
بر اساس این گزارش، به نظر نمی رسد مجموعه داده ما مستقل و به طور یکسان توزیع شده باشد &#8235;(IID). امتیاز کلی &#8235;non-iid برای مجموعه داده (نمایش داده شده در زیر) مربوط به &#8235;`p-value` یک آزمون آماری است که نشان می دهد ترتیب نمونه ها در مجموعه داده مربوط به شباهت بین مقادیر ویژگی آنها است یا خیر. یک &#8235;`p-value` پایین کاملا نشان می‌دهد که مجموعه داده، فرض &#8235;IID را نقض می‌کند، که یک فرض کلیدی برای نتیجه‌گیری (مدل‌های) تولید شده از مجموعه داده برای تعمیم به جمعیت بزرگ‌تر است.

In [ ]:
p_value = lab.get_info('non_iid')['p-value']
p_value

0.0

در اینجا، مجموعه داده ما به‌عنوان &#8235;non-IID علامت شد، زیرا ردیف‌ها بر اساس برچسب کلاس در داده‌های اصلی مرتب شدند. اگر به یاد داشته باشیم که ردیف‌ها را قبل از آموزش مدل و تقسیم داده‌ها مخلوط کنیم، ممکن است این &#8235;(benign) خوش‌خیم باشد. اما اگر نمی‌دانید چرا داده‌های شما به‌عنوان &#8235;non-IID علامت گذاری شده‌اند، باید نگران انحراف احتمالی داده یا تعاملات غیرمنتظره بین نقاط داده باشید (مقادیر آنها ممکن است از نظر آماری مستقل نباشند). به دقت فکر کنید که داده‌های تست آینده ممکن است چگونه باشد (و اینکه آیا داده‌های شما نماینده جمعیتی است که به آن اهمیت می‌دهید یا خیر). شما نباید قبل از اجرای آزمایش non-IID داده های خود را مخلوط کنید (نتیجه گیری آن را باطل می کند).

همانطور که در بالا نشان داده شد، &#8235;cleanlab می‌تواند به‌طور خودکار محتمل‌ترین مسائل موجود در مجموعه داده شما را فهرست کند تا به شما کمک کند مجموعه داده خود را برای مدل‌سازی بعدی مدیریت کنید. با این فهرست کوتاه، می‌توانید تصمیم بگیرید که آیا این مشکلات برچسب را برطرف کنید یا نمونه‌های غیرمعنا یا تکراری را از مجموعه داده خود حذف کنید تا مجموعه داده با کیفیت بالاتری برای آموزش مدل بعدی &#8235;ML خود به دست آورید. تشخیص مشکل &#8235;cleanlab را می توان با خروجی های *هر* نوع مدلی که در ابتدا آموزش داده اید اجرا کرد.

### پروژه متن باز &#8235;Cleanlab

&#8235;[Cleanlab](https://github.com/cleanlab/cleanlab) یک بسته استاندارد هوش مصنوعی مبتنی بر داده است که برای رسیدگی به مسایل کیفیت داده برای داده های کثیف &#8235;(messy) و واقعی طراحی شده است.

به مخزن &#8235;Cleanlab در Github ستاره بدهید، و ما از &#8235;[contribution](https://github.com/cleanlab/cleanlab/issues?q=is:issue+is:open+label:%22good+first+issue%22 ) به پروژه استقبال می کنیم.